In [108]:
import pandas as pd
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_community.vectorstores import chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain import HuggingFacePipeline
from transformers import pipeline
# RAG Chain
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain import HuggingFacePipeline
import transformers
import torch  
import os
from langchain_huggingface import HuggingFaceEmbeddings
import calendar

In [109]:
crop_folders = os.listdir(r".\data")

In [110]:
crop_folders

['Apple', 'Coconut', 'Paddy', 'Potato', 'Tomato']

#### Creating the Dataset from the folder structure

In [ ]:
import os
import pandas as pd

# Path where these folders are located
base_path = r".\data"   # <-- change this to your actual root directory

# Empty list to hold all DataFrames
dfs = []

# Loop through each folder
for folder in crop_folders:
    folder_path = os.path.join(base_path, folder)
    
    # Ensure folder exists
    if os.path.isdir(folder_path):
        # Loop through files in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):   # only process CSV files
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                
                df["disease_name"] = os.path.splitext(file)[0].split(folder + "_")[1].replace("_"," ")
                
                dfs.append(df)

# Concatenate all DataFrames into one big DataFrame
big_df = pd.concat(dfs, ignore_index=True)


In [ ]:
len(big_df)

#### Dropping the answer columns which are empty

In [112]:
big_df = big_df.dropna(subset="KccAns")

#### Getting the Calender names from the month numbers

In [113]:

big_df["Month"] = big_df["Month"].apply(lambda x: calendar.month_name[x])

#### Selecting only those columns to input into the vector DB

In [ ]:
big_df = big_df[["DistrictName","StateName","Season","Month","disease_name","QueryText","KccAns","Crop"]]

#### Getting the word count of the answer column and filtering out which has empty spaces

In [ ]:
big_df["word_count"] = big_df["KccAns"].astype(str).apply(lambda x: len(x.split()))

print(big_df[["KccAns", "word_count"]].head())

In [ ]:
big_df = big_df[big_df["word_count"]>1]

In [ ]:
len(big_df)

In [ ]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display full column width (not truncated)
pd.set_option('display.max_colwidth', None)

#### Dropping all the duplicate rows present in the dataframe

In [ ]:
big_df = big_df.drop_duplicates()

#### Selecting again only those duplicates present for "StateName","QueryText","KccAns"

In [ ]:
big_df= big_df.drop_duplicates(["StateName","QueryText","KccAns"])

#### Filtering the rows which only have numbers in them

In [ ]:
def row_only_digits(row):
    return all(word.isdigit() for word in row.split())

# Apply to the column
big_df = big_df[~big_df["KccAns"].apply(row_only_digits)]

In [ ]:
len(big_df)

#### Getting the word count for the queries

In [ ]:
big_df["query_word_count"] = big_df["QueryText"].astype(str).apply(lambda x: len(x.split()))

# print(big_df[["KccAns", "word_count"]].head())

#### Getting the rows whuch have a word count greater than 2 in the query 

In [ ]:
big_df = big_df[big_df["query_word_count"]>2]

#### Reseting the index

In [ ]:
big_df.reset_index(inplace=True,drop=True)

In [ ]:
big_df.head(2)

In [ ]:
len(big_df)

#### Creating the TF-IDF cosine similarity to filter out rows which are more than 65% similar

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = big_df.copy()

# Create a combined text column
df["combined"] = df["QueryText"].astype(str) + " " + df["KccAns"].astype(str)

# To store filtered rows
filtered_rows = []

# Loop through each crop and disease
for crop in df["Crop"].unique():
    crop_df = df[df["Crop"] == crop]
    for disease in crop_df["disease_name"].unique():
        temp_df = crop_df[crop_df["disease_name"] == disease].copy()
        temp_df = temp_df.reset_index(drop=True)
        
        # Copy of dataframe for comparison
        compare_df = temp_df.copy()
        
        to_keep_indices = []
        
        # Loop through each row in temp_df
        for idx, row in temp_df.iterrows():

            if idx not in compare_df.index:
                continue
            
            combined_text = row["combined"]
            
            if compare_df.empty:
                break
            
            # Compute cosine similarity between current row and all rows in compare_df
            tfidf = TfidfVectorizer()
            texts = [combined_text] + compare_df["combined"].tolist()
            tfidf_matrix = tfidf.fit_transform(texts)
            cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
            
            # Keep current row
            to_keep_indices.append(idx)
            
            # Delete rows in compare_df that are too similar (>0.75) and same exact Q+A
            delete_indices = compare_df.index[(cos_sim > 0.65) |
                                              (compare_df["combined"] == combined_text)]
            compare_df = compare_df.drop(delete_indices)
        
        filtered_rows.append(temp_df.loc[to_keep_indices])

# Combine all filtered data
final_df = pd.concat(filtered_rows).reset_index(drop=True)

print(cos_sim)

# print(final_df)


In [106]:
len(final_df)


53872

#### Saving the csv file

In [ ]:
final_df.to_csv("./filtered_data.csv")

In [ ]:
import matplotlib.pyplot as plt

# Histogram
big_df["Crop"].hist(bins=20, edgecolor="black")

plt.xlabel("Crops")
plt.ylabel("Frequency")
plt.title("Histogram of Crops")
plt.show()